# Usando modelos pré-treinados

Iremos utilizar o Hugging Faces para facilitar a obtenção de modelos pré-treinados para fazermos o transfer learning

In [1]:
!pip install transformers 
!pip install datasets 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 44.5 MB/s 
     |████████████████████████████████| 596 kB 30.9 MB/s 
     |████████████████████████████████| 101 kB 8.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 68.9 MB/s 
     |████████████████████████████████| 1.1 MB 67.3 MB/s 
     |████████████████████████████████| 140 kB 60.2 MB/s 
     |████████████████████████████████| 127 kB 58.3 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 144 kB 46.4 MB/s 
     |██████████████

In [27]:
import torch.nn as nn
import torch

from dataloader import create_CIFAR10_loader
from tqdm import tqdm

In [28]:
#Extraído de https://github.com/rasbt/stat453-deep-learning-ss21/blob/2202699c5fd38af398e2682f289a0868b1b91f0e/L13/code/helper_evaluation.py

def compute_history(model, data_loader, device, loss, n_samples = 50):

    with torch.no_grad():

        loss_ac, correct_pred, num_examples = 0, 0, 0

        for i, batch in enumerate(data_loader):
            features = batch['pixel_values'].to(device)
            targets = batch['labels'].to(device)
            
            output = model(features)

            logits = output['logits']

            loss_val = loss(logits, targets)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            loss_ac += (loss_val.item())
            correct_pred += (predicted_labels == targets.float()).sum()

            if i % n_samples == n_samples - 1:
              break
    return correct_pred.float()/num_examples * 100, loss_ac/num_examples * 100

## Carregando informações pré-treinadas

In [38]:
from transformers import AutoFeatureExtractor, AutoModelForImageClassification

model_name = "google/vit-base-patch16-224"
model_name = "microsoft/resnet-50"

model = AutoModelForImageClassification.from_pretrained(model_name)

In [30]:
model

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [40]:
#model.classifier = nn.Linear(in_features=768, out_features=10, bias=True)
model.classifier[1] = nn.Linear(in_features=2048, out_features=10, bias=True)

In [41]:
model

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [42]:
epochs = 2 
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [62]:
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=10e-3)

In [44]:
train_loader, val_loader = create_CIFAR10_loader(model_name, ID=True)

Files already downloaded and verified
Files already downloaded and verified


In [45]:
for a in train_loader:
  print(a['labels'])
  break

tensor([6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, 9, 9, 3, 2, 6, 4, 3, 6, 6])


In [46]:
minibatch_loss_list, train_acc_list, valid_acc_list, train_loss_list, valid_loss_list = [], [], [], [], []

criterion = torch.nn.functional.cross_entropy

for epoch in range(epochs):

    model.train()
    for batch in tqdm(train_loader):

        optimizer.zero_grad()

        X_train = batch['pixel_values'].to(device)
        y_train = batch['labels'].to(device)

        output = model(X_train)
        logits = output['logits']
        loss = criterion(logits, y_train)
        
        loss.backward()
        optimizer.step()

        minibatch_loss_list.append(loss.item())
        
    #Validação
    model.eval()
    with torch.no_grad():
        train_acc, train_loss = compute_history(model, train_loader, device, criterion)
        valid_acc, valid_loss = compute_history(model, val_loader, device, criterion)
        
        train_acc_list.append(train_acc.item())
        valid_acc_list.append(valid_acc.item())
        
        train_loss_list.append(train_loss)
        valid_loss_list.append(valid_loss)


  2%|▏         | 51/2084 [00:14<09:48,  3.45it/s]


KeyboardInterrupt: ignored

## Congelando algumas camadas para o ajuste da rede

In [47]:
for parameters in model.parameters():
  parameters.requires_grad = False

In [48]:
for p in model.classifier.parameters():
  p.requires_grad = True 

In [63]:
minibatch_loss_list, train_acc_list, valid_acc_list, train_loss_list, valid_loss_list = [], [], [], [], []

criterion = torch.nn.functional.cross_entropy

epochs = 2

for epoch in range(epochs):

    model.train()
    for batch in tqdm(train_loader):

        optimizer.zero_grad()

        X_train = batch['pixel_values'].to(device)
        y_train = batch['labels'].to(device)

        output = model(X_train)
        logits = output['logits']
        loss = criterion(logits, y_train)
        
        loss.backward()
        optimizer.step()

        minibatch_loss_list.append(loss.item())

        
    #Validação
    model.eval()
    with torch.no_grad():
        train_acc, train_loss = compute_history(model, train_loader, device, criterion)
        valid_acc, valid_loss = compute_history(model, val_loader, device, criterion)
        
        train_acc_list.append(train_acc.item())
        valid_acc_list.append(valid_acc.item())
        
        train_loss_list.append(train_loss)
        valid_loss_list.append(valid_loss)


100%|██████████| 2084/2084 [03:01<00:00, 11.51it/s]


In [64]:
print(train_acc_list)

[9.916666984558105, 9.916666984558105]
